In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors
import random
import math
import time
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
import datetime
import operator 
plt.style.use('fivethirtyeight')
%matplotlib inline

In [ ]:
confirmed_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')

In [ ]:
confirmed_cases.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,...,26310,26874,27532,27878,28424,28833,29157,29481,29640,30175,30451,30616,30967,31238,31517,31836,32022,32324,32672,32951,33190,33384,33594,33908,34194,34366,34451,34455,34740,34994,35070,35229,35301,35475,35526,35615,35727,35928,35981,36036
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1672,1722,1788,1838,1891,1962,1995,2047,2114,2192,2269,2330,2402,2466,2535,2580,2662,2752,2819,2893,2964,3038,3106,3188,3278,3371,3454,3571,3667,3752,3851,3906,4008,4090,4171,4290,4358,4466,4570,4637
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,...,11147,11268,11385,11504,11631,11771,11920,12076,12248,12445,12685,12968,13273,13571,13907,14272,14657,15070,15500,15941,16404,16879,17348,17808,18242,18712,19195,19689,20216,20770,21355,21948,22549,23084,23691,24278,24872,25484,26159,26764
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,854,854,855,855,855,855,855,855,855,855,855,855,855,855,855,855,855,855,855,855,855,855,855,855,855,855,855,858,861,862,877,880,880,880,884,884,889,889,897,897
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,148,155,166,172,176,183,186,189,197,212,212,259,267,276,284,291,315,328,346,346,346,386,386,396,458,462,506,525,541,576,607,638,687,705,749,779,812,851,880,916


In [ ]:
deaths_reported = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')

In [ ]:
deaths_reported.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,491,504,546,548,569,581,598,618,639,675,683,703,721,733,746,774,807,819,826,864,898,920,936,957,971,994,1010,1012,1048,1094,1113,1147,1164,1181,1185,1186,1190,1211,1225,1248
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,37,38,39,42,43,44,44,45,47,49,51,53,55,58,62,65,69,72,74,76,79,81,83,83,85,89,93,95,97,101,104,107,111,112,113,117,120,123,128,134
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,788,799,811,825,837,845,852,861,869,878,885,892,897,905,912,920,928,937,946,952,959,968,978,988,996,1004,1011,1018,1028,1040,1052,1057,1068,1078,1087,1100,1111,1124,1136,1146
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,6,7,8,8,9,9,10,10,10,10,10,10,11,11,13,15,17,18,19,19,19,21,21,22,23,23,26,26,26,27,28,29,29,29,29,30,33,33,35,39


In [ ]:

recovered_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

In [ ]:
recovered_cases.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,5508,6158,7660,7962,8292,8764,8841,9260,9869,10174,10306,10674,12604,13934,14131,15651,16041,17331,19164,19366,20103,20179,20700,20847,20882,21135,21216,21254,21454,22456,22824,23151,23273,23634,23741,23741,23924,24550,24602,24793
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1064,1077,1086,1114,1126,1134,1159,1195,1217,1250,1298,1346,1384,1438,1459,1516,1559,1592,1637,1657,1702,1744,1791,1832,1875,1881,1946,2014,2062,2091,2137,2214,2264,2311,2352,2397,2463,2523,2608,2637
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,7842,7943,8078,8196,8324,8422,8559,8674,8792,8920,9066,9202,9371,9674,9897,10040,10342,10832,11181,11492,11884,12094,12329,12637,13124,13124,13743,14019,14295,14792,15107,15430,15744,16051,16400,16646,16983,17369,17369,18076
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,789,791,792,792,792,792,796,797,797,797,799,799,799,799,799,799,800,800,800,800,800,800,802,802,803,803,803,803,803,803,803,803,803,803,803,803,803,803,803,803
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,64,64,64,66,66,77,77,77,77,81,81,81,81,93,93,97,97,107,108,108,108,117,117,117,117,118,118,118,118,124,124,199,210,221,221,221,221,236,241,242


In [ ]:
# Fetching all the columns from confirmed dataset
cols = confirmed_cases.keys()
date = []
i=0
for col in confirmed_cases:
  if i>3:
    date.append(col)
  i=i+1
print(date)

['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20', '1/28/20', '1/29/20', '1/30/20', '1/31/20', '2/1/20', '2/2/20', '2/3/20', '2/4/20', '2/5/20', '2/6/20', '2/7/20', '2/8/20', '2/9/20', '2/10/20', '2/11/20', '2/12/20', '2/13/20', '2/14/20', '2/15/20', '2/16/20', '2/17/20', '2/18/20', '2/19/20', '2/20/20', '2/21/20', '2/22/20', '2/23/20', '2/24/20', '2/25/20', '2/26/20', '2/27/20', '2/28/20', '2/29/20', '3/1/20', '3/2/20', '3/3/20', '3/4/20', '3/5/20', '3/6/20', '3/7/20', '3/8/20', '3/9/20', '3/10/20', '3/11/20', '3/12/20', '3/13/20', '3/14/20', '3/15/20', '3/16/20', '3/17/20', '3/18/20', '3/19/20', '3/20/20', '3/21/20', '3/22/20', '3/23/20', '3/24/20', '3/25/20', '3/26/20', '3/27/20', '3/28/20', '3/29/20', '3/30/20', '3/31/20', '4/1/20', '4/2/20', '4/3/20', '4/4/20', '4/5/20', '4/6/20', '4/7/20', '4/8/20', '4/9/20', '4/10/20', '4/11/20', '4/12/20', '4/13/20', '4/14/20', '4/15/20', '4/16/20', '4/17/20', '4/18/20', '4/19/20', '4/20/20', '4/21/20', '4/22/20', '4/23/20', '4/

In [ ]:
# Extracting the date columns
confirmed = confirmed_cases.loc[:, cols[4]:cols[-1]]
deaths = deaths_reported.loc[:, cols[4]:cols[-1]]
recoveries = recovered_cases.loc[:, cols[4]:cols[-1]]

In [ ]:
confirmed

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,...,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,...,26310,26874,27532,27878,28424,28833,29157,29481,29640,30175,30451,30616,30967,31238,31517,31836,32022,32324,32672,32951,33190,33384,33594,33908,34194,34366,34451,34455,34740,34994,35070,35229,35301,35475,35526,35615,35727,35928,35981,36036
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1672,1722,1788,1838,1891,1962,1995,2047,2114,2192,2269,2330,2402,2466,2535,2580,2662,2752,2819,2893,2964,3038,3106,3188,3278,3371,3454,3571,3667,3752,3851,3906,4008,4090,4171,4290,4358,4466,4570,4637
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,...,11147,11268,11385,11504,11631,11771,11920,12076,12248,12445,12685,12968,13273,13571,13907,14272,14657,15070,15500,15941,16404,16879,17348,17808,18242,18712,19195,19689,20216,20770,21355,21948,22549,23084,23691,24278,24872,25484,26159,26764
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,854,854,855,855,855,855,855,855,855,855,855,855,855,855,855,855,855,855,855,855,855,855,855,855,855,855,855,858,861,862,877,880,880,880,884,884,889,889,897,897
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,148,155,166,172,176,183,186,189,197,212,212,259,267,276,284,291,315,328,346,346,346,386,386,396,458,462,506,525,541,576,607,638,687,705,749,779,812,851,880,916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,671,683,688,693,698,698,702,707,710,711,712,713,713,713,714,715,717,719,719,720,721,724,724,726,727,727,729,732,732,737,740,741,743,746,746,746,747,749,860,862
262,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,885,902,909,919,922,941,967,992,1015,1076,1089,1103,1118,1128,1158,1190,1221,1240,1248,1265,1284,1297,1318,1356,1380,1389,1465,1498,1516,1526,1552,1576,1581,1606,1619,1629,1640,1654,1674,1674
263,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,197,197,210,210,247,247,247,265,265,272,272,272,272,272,303,303,303,309,309,311,311,311,313,314,314,317,317,317,321,321,328,328,328,334,334,337,337,340,340,340
264,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,5160,5221,5279,5338,5399,5457,5513,5567,5630,5691,5747,5799,5849,5900,5900,6005,6058,6058,6159,6213,6262,6315,6364,6410,6457,6506,6552,6596,6643,6695,6741,6786,6834,6878,6921,6967,7015,7060,7104,7150


In [ ]:
dates = confirmed.keys()

In [ ]:
us_cases = []
us_deaths = []
mortality_rate = []
recovery_rate = []
us_recoveries = []

for i in dates:
  confirmed_sum = confirmed_cases[confirmed_cases['Country/Region']=='US'][i].sum()
  death_sum = deaths_reported[deaths_reported['Country/Region']=='US'][i].sum()
  recovery_sum = recovered_cases[recovered_cases['Country/Region']=='US'][i].sum()

  #mortality rate
  mortality_rate.append(death_sum/confirmed_sum*100)
  #recovery rate
  recovery_rate.append(recovery_sum/confirmed_sum*100)
  #total cases in US
  us_cases.append(confirmed_cases[confirmed_cases['Country/Region']=='US'][i].sum())
  #total deaths in US
  us_deaths.append(deaths_reported[deaths_reported['Country/Region']=='US'][i].sum())
  #total recoveries in US
  us_recoveries.append(recovered_cases[recovered_cases['Country/Region']=='US'][i].sum())
  


In [ ]:
mortality_rate

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 4.166666666666666,
 3.3333333333333335,
 11.320754716981133,
 9.58904109589041,
 10.576923076923077,
 6.896551724137931,
 6.306306306306306,
 5.044510385756676,
 4.656319290465632,
 4.238921001926782,
 3.938115330520394,
 2.9756537421100093,
 2.754644458680333,
 2.410755679184052,
 2.0209059233449476,
 2.358490566037736,
 2.224770642201835,
 2.133203061390653,
 2.1083323987888303,
 1.8718655082291444,
 1.8584116227732432,
 1.765730880929332,
 1.780160540268357,
 1.7879133409350056,
 1.886827321111768,
 2.018015290288396,
 2.076322079651806,
 2.2488169267472333,
 2.40355864306253,
 2.521865373039198,
 2.692570079959682,
 2.96994552892054,
 3.196003828108587,
 3.4471199051551453,
 3.5245365163968514,
 3.5618455338893575,
 3.6940158128760294,
 3.850060591206786,
 

In [ ]:
def daily_increase(data):
    d = [] 
    for i in range(len(data)):
        if i == 0:
            d.append(data[0])
        else:
            d.append(data[i]-data[i-1])
    return d

In [ ]:
#daily increase in cases
us_daily_increase = daily_increase(us_cases)

In [ ]:
us_daily_increase

[1,
 0,
 1,
 0,
 3,
 0,
 0,
 0,
 0,
 2,
 1,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 8,
 6,
 23,
 20,
 31,
 70,
 48,
 115,
 114,
 68,
 192,
 398,
 452,
 596,
 713,
 98,
 1392,
 1781,
 2776,
 5240,
 5322,
 6346,
 7936,
 10089,
 10262,
 11943,
 18036,
 18185,
 19793,
 19136,
 21502,
 26017,
 25481,
 30405,
 31937,
 33152,
 27874,
 29642,
 30777,
 31694,
 34756,
 33501,
 30026,
 28553,
 25291,
 27065,
 29096,
 31298,
 32724,
 28341,
 26038,
 27341,
 25602,
 28104,
 34195,
 36291,
 32921,
 27689,
 22465,
 24535,
 27520,
 29629,
 34162,
 29195,
 25587,
 22475,
 24185,
 25256,
 27882,
 27178,
 25733,
 19764,
 18878,
 22190,
 20958,
 27617,
 25300,
 25101,
 19004,
 21769,
 20449,
 23807,
 25355,
 24141,
 21823,
 20813,
 18991,
 18883,
 18282,
 22815,
 24504,
 24450,
 19910,
 17355,
 20895,
 19958,
 21351,
 25224,
 22732,
 17731,
 17415,
 18127,
 20794,
 22950,
 25330,
 25556,
 19824,
 19660,
 23705,
 25559,
 27809,
 31480,
 3

In [ ]:
#confirmed cases in US
us_cases

[1,
 1,
 2,
 2,
 5,
 5,
 5,
 5,
 5,
 7,
 8,
 8,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 12,
 12,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 15,
 15,
 15,
 15,
 15,
 15,
 16,
 16,
 24,
 30,
 53,
 73,
 104,
 174,
 222,
 337,
 451,
 519,
 711,
 1109,
 1561,
 2157,
 2870,
 2968,
 4360,
 6141,
 8917,
 14157,
 19479,
 25825,
 33761,
 43850,
 54112,
 66055,
 84091,
 102276,
 122069,
 141205,
 162707,
 188724,
 214205,
 244610,
 276547,
 309699,
 337573,
 367215,
 397992,
 429686,
 464442,
 497943,
 527969,
 556522,
 581813,
 608878,
 637974,
 669272,
 701996,
 730337,
 756375,
 783716,
 809318,
 837422,
 871617,
 907908,
 940829,
 968518,
 990983,
 1015518,
 1043038,
 1072667,
 1106829,
 1136024,
 1161611,
 1184086,
 1208271,
 1233527,
 1261409,
 1288587,
 1314320,
 1334084,
 1352962,
 1375152,
 1396110,
 1423727,
 1449027,
 1474128,
 1493132,
 1514901,
 1535350,
 1559157,
 1584512,
 1608653,
 1630476,
 1651289,
 1670280,
 1689163,
 1707445,
 1730260,
 1754764,
 1779214,
 1799124,
 1816479,
 1

In [ ]:
#daily deaths report
us_daily_death = daily_increase(us_deaths)
us_daily_death

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 5,
 1,
 4,
 1,
 2,
 3,
 4,
 1,
 6,
 5,
 10,
 9,
 6,
 12,
 27,
 34,
 57,
 77,
 97,
 94,
 145,
 183,
 237,
 312,
 413,
 554,
 634,
 627,
 820,
 1224,
 1241,
 1586,
 1315,
 1284,
 1439,
 1668,
 2309,
 2116,
 2075,
 2093,
 2046,
 1771,
 1828,
 2404,
 2549,
 2113,
 2614,
 2386,
 1229,
 1795,
 2457,
 2389,
 2437,
 2118,
 1650,
 1293,
 1425,
 2219,
 2506,
 2296,
 1869,
 1556,
 1096,
 1309,
 2307,
 2458,
 1919,
 1727,
 1479,
 879,
 1013,
 1615,
 1751,
 1774,
 1661,
 1223,
 752,
 770,
 1551,
 1568,
 1223,
 1269,
 1121,
 608,
 513,
 685,
 1509,
 1182,
 1166,
 969,
 586,
 772,
 1033,
 990,
 1009,
 970,
 680,
 451,
 505,
 940,
 917,
 881,
 822,
 750,
 298,
 391,
 839,
 751,
 704,
 672,
 610,
 255,
 406,
 837,
 757,
 2422,
 605,
 489,
 241,
 350,
 721,
 673,
 698,
 639,
 247,
 271,
 325,
 1195,
 820,
 990,
 811,
 676,
 428,
 361,
 900,
 949

In [ ]:
#daily recovery report
us_daily_recovery = daily_increase(us_recoveries)
us_daily_recovery

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 4,
 0,
 0,
 0,
 5,
 0,
 88,
 16,
 26,
 29,
 2,
 0,
 170,
 13,
 320,
 188,
 203,
 1593,
 2979,
 1380,
 1450,
 527,
 706,
 4945,
 2796,
 2133,
 2182,
 1796,
 1851,
 3380,
 2480,
 1718,
 10494,
 4281,
 4333,
 2607,
 3842,
 6295,
 5497,
 1992,
 2875,
 2162,
 2837,
 18876,
 1293,
 6616,
 4436,
 4512,
 4784,
 33227,
 10068,
 11367,
 4770,
 7028,
 2611,
 119,
 5126,
 3957,
 13541,
 3635,
 16564,
 -2446,
 13143,
 2984,
 4333,
 17629,
 3889,
 10913,
 6214,
 4920,
 4106,
 51717,
 11104,
 5497,
 12421,
 5745,
 6606,
 8483,
 6455,
 10015,
 28297,
 13473,
 5637,
 15390,
 5744,
 6704,
 9143,
 5518,
 12155,
 6333,
 8649,
 6788,
 7094,
 9220,
 5210,
 14518,
 7169,
 8688,
 6924,
 7600,
 10745,
 4673,
 18065,
 7350,
 8613,
 7401,
 7247,
 8499,
 5856,
 20039,
 15428,
 9363,
 51976,
 8434,
 103921,
 12438,
 

In [ ]:

dataset = pd.DataFrame({'date':date,'deaths':us_deaths,'recovery':us_recoveries,'daily_cases_increase':us_daily_increase,'daily_recovery':us_daily_recovery,'daily_death':us_daily_death,'mortality_rate':mortality_rate,'recovery_rate':recovery_rate,'Confirmed_cases':us_cases,})
dataset.to_csv('final_dataset.csv',index=False)

In [ ]:
dataset=pd.read_csv('final_dataset.csv')

In [ ]:
dataset.head(10)
# training_set = dataset.iloc[:,1:8].values

,date,deaths,recovery,daily_cases_increase,daily_recovery,daily_death,mortality_rate,recovery_rate,Confirmed_cases
0,1/22/20,0,0,1,0,0,0.0,0.0,1
1,1/23/20,0,0,0,0,0,0.0,0.0,1
2,1/24/20,0,0,1,0,0,0.0,0.0,2
3,1/25/20,0,0,0,0,0,0.0,0.0,2
4,1/26/20,0,0,3,0,0,0.0,0.0,5
5,1/27/20,0,0,0,0,0,0.0,0.0,5
6,1/28/20,0,0,0,0,0,0.0,0.0,5
7,1/29/20,0,0,0,0,0,0.0,0.0,5
8,1/30/20,0,0,0,0,0,0.0,0.0,5
9,1/31/20,0,0,2,0,0,0.0,0.0,7


In [ ]:
# training_set
# import random
# random.shuffle(training_set)
# X_test = training_set[-20:]
# training_set = training_set[:-20]
# print(training_set.shape)
# # print(X_test)
# X_test_final = []
# j=60
# for i in range (0,20):
#   X_test_final.append(X_test[i:j,0])
#   j=j+1

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# sc = MinMaxScaler(feature_range = (0, 1))
# training_set_scaled = sc.fit_transform(training_set)
# print(training_set_scaled.shape)

In [ ]:
# X_train = []
# y_train = []
# for i in range(60, 166):
#     X_train.append(training_set_scaled[i-60:i, 0])
#     y_train.append(training_set_scaled[i, 0])
    
#     # X_train.append(training_set[i-60:i, 0])
#     # y_train.append(training_set[i, 0])
# X_train, y_train = np.array(X_train), np.array(y_train)
# X_test_final = np.array(X_test_final)
# X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
# X_test_final = np.reshape(X_test_final, (X_test_final.shape[0],X_test_final.shape[1],1))
# x_val = X_train[-10:]
# y_val = y_train[-10:]
# X_train = X_train[:-10]
# y_train = y_train[:-10]
# # print(X_train)

In [ ]:
# import torch

# import os
# import numpy as np
# import pandas as pd
# from tqdm import tqdm
# import seaborn as sns
# from pylab import rcParams
# import matplotlib.pyplot as plt
# from matplotlib import rc
# from sklearn.preprocessing import MinMaxScaler
# from pandas.plotting import register_matplotlib_converters
# from torch import nn, optim

# %matplotlib inline

In [ ]:
# test_data_size = 14

# train_data = dataset[:-test_data_size]
# test_data = dataset[-test_data_size:]

# train_data.shape

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# sc = MinMaxScaler(feature_range = (0, 1))
# train_data = sc.fit_transform(train_data)
# print(train_data.shape)

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# sc = MinMaxScaler(feature_range = (0, 1))
# test_data = sc.fit_transform(test_data)
# print(test_data.shape)

In [ ]:
# def create_sequences(data, seq_length):
#     xs = []
#     ys = []

#     for i in range(len(data)-seq_length-1):
#         x = data[i:(i+seq_length)]
#         y = data[i+seq_length]
#         xs.append(x)
#         ys.append(y)

#     return np.array(xs), np.array(ys)

In [ ]:
# seq_length = 5
# # X_train = train_data[:,:-1]
# # y_train = train_data[:,-1]
# # X_test = test_data[:,test_data.columns!='Confirmed_cases']
# # y_test = test_data[:,-1]
# X_train, y_train = create_sequences(train_data, seq_length)
# X_test, y_test = create_sequences(test_data, seq_length)
# # X_train = np.array(X_train)
# # y_train = np.array(y_train)
# # X_test = np.array(X_test)
# # y_test = np.array(y_test)

# X_train = torch.from_numpy(X_train).float()
# y_train = torch.from_numpy(y_train).float()

# X_test = torch.from_numpy(X_test).float()
# y_test = torch.from_numpy(y_test).float()

In [ ]:
# X_train[:2]

In [ ]:
# y_train[:-2]

In [ ]:
# class CoronaVirusPredictor(nn.Module):

#   def __init__(self, n_features, n_hidden, seq_len, n_layers=2):
#     super(CoronaVirusPredictor, self).__init__()

#     self.n_hidden = n_hidden
#     self.seq_len = seq_len
#     self.n_layers = n_layers

#     self.lstm = nn.LSTM(
#       input_size=n_features,
#       hidden_size=n_hidden,
#       num_layers=n_layers,
#       dropout=0.5
#     )

#     self.linear = nn.Linear(in_features=n_hidden, out_features=1)

#   def reset_hidden_state(self):
#     self.hidden = (
#         torch.zeros(self.n_layers, self.seq_len, self.n_hidden),
#         torch.zeros(self.n_layers, self.seq_len, self.n_hidden)
#     )

#   def forward(self, sequences):
#     lstm_out, self.hidden = self.lstm(
#       sequences.view(len(sequences), self.seq_len, -1),
#       self.hidden
#     )
#     last_time_step = \
#       lstm_out.view(self.seq_len, len(sequences), self.n_hidden)[-1]
#     y_pred = self.linear(last_time_step)
#     return y_pred

In [ ]:
# def train_model(
#   model, 
#   train_data, 
#   train_labels, 
#   test_data=None, 
#   test_labels=None
# ):
#   loss_fn = torch.nn.MSELoss(reduction='sum')

#   optimiser = torch.optim.Adam(model.parameters(), lr=1e-3)
#   num_epochs = 60

#   train_hist = np.zeros(num_epochs)
#   test_hist = np.zeros(num_epochs)

#   for t in range(num_epochs):
#     model.reset_hidden_state()

#     y_pred = model(X_train)

#     loss = loss_fn(y_pred.float(), y_train)

#     if test_data is not None:
#       with torch.no_grad():
#         y_test_pred = model(X_test)
#         test_loss = loss_fn(y_test_pred.float(), y_test)
#       test_hist[t] = test_loss.item()

#       if t % 10 == 0:  
#         print(f'Epoch {t} train loss: {loss.item()} test loss: {test_loss.item()}')
#     elif t % 10 == 0:
#       print(f'Epoch {t} train loss: {loss.item()}')

#     train_hist[t] = loss.item()
    
#     optimiser.zero_grad()

#     loss.backward()

#     optimiser.step()
  
#   return model.eval(), train_hist, test_hist

In [ ]:
# model = CoronaVirusPredictor(
#   n_features=1, 
#   n_hidden=512, 
#   seq_len=seq_length, 
#   n_layers=2
# )
# model, train_hist, test_hist = train_model(
#   model, 
#   X_train, 
#   y_train, 
#   X_test, 
#   y_test
# )